In [1]:
![image.png](attachment:image.png)

'[image.png]' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
![image.png](attachment:image.png) ![image-2.png](attachment:image-2.png)

'[image.png]' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
# definitions

beam = {
    "Area"     : 9880  , #mm2
    "d"        : 450   , #mm
    "h"        : 378.8 , #mm
    "b_t"      : 190   , #mm
    "t_flange" : 14.6  , #mm
    "t_web"    : 9.4   , #mm
    "I_x"      : 33740 , #cm4
}

sac  = {
    "h_r"   : 50, #mm görseldeki h, hadve yüksekliği
    "w_r"   : 165, #mm
    "t_sac" : 1.2 #mm
}

anchorage = {
    "r_f" : 19, #mm
    "h_a" : 100, #mm
    "Fu"  : 450 #N/mm2
}

f_ck = 30 #N/mm2
Fy = 355 #N/mm2
Fu = 510 #N/mm2
Pg = 28 #kN
Pq = 10.5 #kN
L = 9 #m
wg = 1. #kN/m
h_c_üs = 80 #mm net beton örtüsü
h_c = h_c_üs + sac["h_r"]